In [3]:
import csv
import json
import os
import struct
import wave
import matplotlib.pyplot as plt
import numpy as np

__version__ = '1.2.0'

In [4]:
def load_data(filepath, cw):
    if cw == 1:
        return load_chest_data(filepath)
    else:
        return load_wrist_data(filepath)

def load_chest_data(filepath, start=0):
    """
    Loads a  file according to the specified format. Time t=0 corresponds to
    the beginning of the record.
    :param str filepath:
    :param int start: an offset in second to add to the time of the data
    :return: list
    """
    data = []
    if os.path.exists(filepath + '.wav'):
        data = load_wave(filepath + '.wav', start)
    elif os.path.exists(filepath + '.csv'):
        data = load_csv(filepath + '.csv', start)
    if os.path.exists(filepath) and filepath.endswith('.wav'):
        data = load_wave(filepath, start)
    elif os.path.exists(filepath) and filepath.endswith('.csv'):
        data = load_csv(filepath, start)
    return data

def load_wrist_data(filepath):
    data = []
    filename = os.path.splitext(os.path.split(filepath)[1])[0]
    with open(filepath, 'r') as csvfile:
        reader_csv = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader_csv:
            try:
                if (filename == 'ACC'):
                    t = (float(row[0]), float(row[1]) , float(row[2]))
                    data.append(t)
                else:
                    data.append(float(row[0]))
            except ValueError:
                pass
    return data


def load_csv(filepath, start=0):
    """
    Loads a csv file according to the specified format. Time t=0 corresponds to
    the beginning of the record.
    :param str filepath:
    :param int start: an offset in second to add to the time of the data
    :return: np.array
    """
    data = []
    file_specs = getspec(filepath)
    with open(filepath, 'r') as csvfile:
        reader_csv = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader_csv:
            try:
                try:
                    int(row[1])  # ensure it is an int otherwise is was already converted, don't change second column
                    data.append([float(row[0]) + start, float(row[1]) * file_specs['gain'] + file_specs['offset']])
                except ValueError:
                    data.append([float(row[0]) + start, float(row[1])])
            except ValueError:
                pass
    return data


def load_wave(filepath, start=0):
    """
    Loads a WAV file according to the specified format. Time t=0 corresponds to
    the beginning of the record.
    :param str filepath:
    :param int start: an offset in second to add to the time of the data
    :return:
    """
    # note data can be loaded with scipy.io.wavfileor numpy, but we keep here a decoding possible
    # with the basic python package
    file_specs = getspec(filepath)
    f = wave.open(filepath)
    if file_specs['signed']:
        yy = struct.unpack("h" * f.getnframes(), f.readframes(f.getnframes()))
    else:
        yy = struct.unpack("H" * f.getnframes(), f.readframes(f.getnframes()))
    f.close()
    freq_signal = f.getframerate()
    if 1 / freq_signal == file_specs['dt']:
        dt = file_specs['dt']
    elif freq_signal == file_specs['dt']:
        dt = file_specs['dt']
    else:
        raise ('invalid frequency {}'.format(filepath))

    return [(float(t) * dt + file_specs['dt_offset'] + start,
             float(y) * file_specs['gain'] + file_specs['offset']) for t, y in enumerate(yy)]


def get_files_specs(device_freq, as_dict=True):
    """
    Get all specs for a given frequency
    :param device_freq: device frequency 256 or 1000
    :return: table or dict of all available datatype
    """
    GAIN_ECG = 0.0064 if device_freq == 256 else 0.001
    OFFSET_ECG = -1360 * 0.0064 if device_freq == 256 else 0
    dt_e = 1 / 256 if device_freq == 256 else 1 / 250
    dt_r = 1 / 128 if device_freq == 256 else 1 / 125
    dt_a = 1 / 64 if device_freq == 256 else 1 / 50
    dt_o = 1 / 64 if device_freq == 256 else 1 / 75
    br_gain = 1 if device_freq == 256 else .1
    gain_resp = 13.28 if device_freq == 256 else 1
    tmp_gain = 1 if device_freq == 256 else 1 / 256
    keys_list = ['filename', 'filename_out', 'quality_signal_name', 'unit', 'dt', 'dt_offset',
                 'gain', 'offset', 'signed', 'round', 'datatype', 'is_wav']
    specs = [
        # filename, csv_name, quality_signal_name, unit, dt, dt_offset, gain, offset, signed, round, datatype, is_wav
        ["ECG_I", "ECG_I", "", "mV", dt_e, 0, GAIN_ECG, OFFSET_ECG, 1, 5, 4113, True],
        ["acceleration_X", "acceleration_X", "", "G", dt_a, 0, 1 / 256, 0, 1, 8, 4145, True],
        ["acceleration_Y", "acceleration_Y", "", "G", dt_a, 0, 1 / 256, 0, 1, 8, 4146, True],
        ["acceleration_Z", "acceleration_Z", "", "G", dt_a, 0, 1 / 256, 0, 1, 8, 4147, True],
        ["breathing_rate", "breathing_rate", "breathing_rate_quality", "rpm", 1, 1, br_gain, 0, 0, 0, 33, True],
        ["RR_interval", "RR_interval_epoch", "RR_interval_quality", "s", 1, 0, 1 / device_freq, 0, 0, 8, 18, False],
        ["inspiration", "inspiration_epoch", "", "na", 1, 0, 1, 0, 0, 0, 34, False],
        ["expiration", "expiration_epoch", "", "", 1, 0, 1, 0, 0, 0, 35, False],
        ["NN_interval", "NN_interval_epoch", "", "s", 1, 0, 1 / device_freq, 0, 0, 8, 318, False],
    ]
    if as_dict:
        return {vals[0]: {key: val for key, val in zip(keys_list, vals)} for vals in specs}
    else:
        return specs


def getspec(filepath):
    """
    return the specs of the given file
    :param str filepath:
    :return: dict
    """
    filename = os.path.splitext(os.path.split(filepath)[1])[0]
    info = load_info(os.path.dirname(filepath))
    device_freq = info['freq']
    try:
        return get_files_specs(device_freq)[filename]
    except ValueError:
        raise AssertionError('Not a valid filename {}'.format(filepath))


def load_info(directory):
    """
    load the info.json file
    :param str directory:
    :return: dict
     """
    with open(os.path.join(directory, "info.json"), "r") as f:
        info = json.load(f)
        info['freq'] = _check_freq(info)
    return info


def _check_freq(info):
    """Check the frequency of the data.
    This allow to load data from hexoskin (256 Hz) and Astroskin (1000 Hz).
    :param dict info:
    :return: frequency
    """

    root_start = info.get('start', info.get('start_timestamp', 0))
    if 1230768000 * 256 < root_start < 2524608000 * 256:  # 2009, 2050
        return 256
    elif 1230768000 * 1000 < root_start < 2524608000 * 1000:  # 2009, 2050
        return 1000
    else:
        raise AssertionError("start_timestamp ={}. out of range. Aborted.".format(root_start))

In [5]:
def open_file(file_name, cw_file_name, cwd, cw):
    file_path = cwd + '\\' + file_name + '\\' + cw_file_name
    data = load_data(file_path, cw)
    return data


def data_to_dict(curdir, cwd, cw_file_names, start_file, cw):
    files = os.listdir(curdir)
    file_names = [f for f in files if start_file in f]
    dicts = [{os.path.splitext(cw_file_name)[0] : open_file(file_name, cw_file_name, cwd, cw) for cw_file_name in cw_file_names} for file_name in file_names]
    return dicts

In [6]:
def update_acc_in_dict(record_dict):
    n = len(record_dict['acceleration_X'])
    acc = np.zeros((n, 3))
    acc[:, 0] =  np.asarray(list(list(zip(*(record_dict['acceleration_X'])))[1]))
    acc[:, 1] =  np.asarray(list(list(zip(*(record_dict['acceleration_Y'])))[1]))
    acc[:, 2] =  np.asarray(list(list(zip(*(record_dict['acceleration_Z'])))[1]))
    record_dict['ACC'] = acc
    record_dict.pop('acceleration_X')
    record_dict.pop('acceleration_Y')
    record_dict.pop('acceleration_Z')
    
def update_ECG_in_dict(record_dict):
    n = len(record_dict['ECG_I'])
    ecg = np.zeros((n, 1))
    ecg[:, 0] = np.asarray(list(list(zip(*(record_dict['ECG_I'])))[1]))
    record_dict['ECG'] = ecg
    record_dict.pop('ECG_I')

def update_resp_in_dict(record_dict):
    n = len(record_dict['breathing_rate'])
    resp = np.zeros((n, 1))
    resp[:, 0] = np.asarray(list(list(zip(*(record_dict['breathing_rate'])))[1]))
    record_dict['Resp'] = resp
    record_dict.pop('breathing_rate')
    
def update_insp_expi_dict(record_dict):
    ne = len(record_dict['expiration'])
    ni = len(record_dict['inspiration'])
    expi = np.zeros((ne, 2))
    insp = np.zeros((ni, 2))
    expi[:, 0] = np.asarray(list(list(zip(*(record_dict['expiration'])))[0]))
    expi[:, 1] = np.asarray(list(list(zip(*(record_dict['expiration'])))[1]))
    insp[:, 0] = np.asarray(list(list(zip(*(record_dict['inspiration'])))[0]))
    insp[:, 1] = np.asarray(list(list(zip(*(record_dict['inspiration'])))[1]))
    record_dict["expiration"] = expi
    record_dict["inspiration"] = insp

def update_NN_RR(record_dict):
    n = len(record_dict['NN_interval'])
    r = len(record_dict['RR_interval'])
    nn = np.zeros((n, 2))
    rr = np.zeros((r, 2))
    nn[:, 0] = np.asarray(list(list(zip(*(record_dict['NN_interval'])))[0]))
    nn[:, 1] = np.asarray(list(list(zip(*(record_dict['NN_interval'])))[1]))
    rr[:, 0] = np.asarray(list(list(zip(*(record_dict['RR_interval'])))[0]))
    rr[:, 1] = np.asarray(list(list(zip(*(record_dict['RR_interval'])))[1]))
    record_dict["NN_interval"] = nn
    record_dict["RR_interval"] = rr
    
def update_dict(record_dict):
    update_acc_in_dict(record_dict)
    update_ECG_in_dict(record_dict)
    update_resp_in_dict(record_dict)
    update_insp_expi_dict(record_dict)
    update_NN_RR(record_dict)

In [7]:
def update_acc_wrist(record_dict):
    n = len(record_dict['ACC'])
    acc = np.zeros((n, 3))
    x = list(zip(*(record_dict['ACC'])))
    acc[:, 0] =  np.asarray(list(x[0]))
    acc[:, 1] =  np.asarray(list(x[1]))
    acc[:, 2] =  np.asarray(list(x[2]))
    record_dict['ACC'] = acc
    
def update_BVP(record_dict):
    n = len(record_dict['BVP'])
    bvp = np.zeros((n, 1))
    bvp[:, 0] = np.asarray(record_dict['BVP'])
    record_dict['BVP'] = bvp

def update_EDA(record_dict):
    n = len(record_dict['EDA'])
    eda = np.zeros((n, 1))
    eda[:, 0] = np.asarray(record_dict['EDA'])
    record_dict['EDA'] = eda
    
def update_TEMP(record_dict):
    n = len(record_dict['TEMP'])
    tem = np.zeros((n, 1))
    tem[:, 0] = np.asarray(record_dict['TEMP'])
    record_dict['TEMP'] = tem
    
def update_wdict(record_dict):
    update_acc_wrist(record_dict)
    update_BVP(record_dict)
    update_EDA(record_dict)
    update_TEMP(record_dict)

In [8]:
chest_file_names = ['acceleration_X.wav', 
              'acceleration_Y.wav', 
              'acceleration_Z.wav', 
              'breathing_rate.wav', 
              'ECG_I.wav', 
              'expiration.csv', 
              'inspiration.csv', 
              'NN_interval.csv', 
              'RR_interval.csv']
dicts = data_to_dict(os.curdir, os.getcwd(), chest_file_names, 'record', 1)
# change back to regular list of tuples instead of pd df???

In [9]:
for d in dicts:
    update_dict(d)

In [10]:
wrist_file_names = ['ACC.csv',
                    'BVP.csv',
                    'EDA.csv',
                    'TEMP.csv']
wdicts = data_to_dict(os.curdir, os.getcwd(), wrist_file_names, 'A037F7', 0)

In [11]:
for d in wdicts:
    update_wdict(d)

In [12]:
wesad = {}
for i in range(len(dicts)):
    signal = {}
    signal['chest'] = dicts[i]
    signal['wrist'] = wdicts[i]
    wesad['signal'] = signal
    
z = np.zeros(100)
wesad['label'] = z
wesad['subject'] = 'Participant_7'

In [15]:
wesad['signal'].keys()

dict_keys(['chest', 'wrist'])

In [18]:
wesad['signal']['chest'].keys()

dict_keys(['expiration', 'inspiration', 'NN_interval', 'RR_interval', 'ACC', 'ECG', 'Resp'])

In [19]:
wesad['signal']['wrist'].keys()

dict_keys(['ACC', 'BVP', 'EDA', 'TEMP'])

In [14]:
# Ordering of Record and A03 files: How do they match?
# Do I need to take acc norm
#